In [2]:
import sys
import os
%matplotlib inline

In [3]:
# get current working directory
os.getcwd()
# change directory
os.chdir('../')
os.getcwd()

'c:\\Users\\gtheis2\\git\\ACTC_MD_ML'

In [4]:
from rdkit.Chem.Draw import rdMolDraw2D
from src.trainers import Trainers
from src.polymerSMILES import polymers
from src.dataProcessing import * #DataPreprocessing #computeTanimotoSimilarity #* #data_load, computeTanimotoSimilarity, get_dissimilarity_map, featurizer, normalizedata
from src.solver import *

In [5]:
# load smiles
polymers_dict = polymers.get_data() # get rdkit formats of the polymer smiles 

In [6]:
# instantiate data preprocessing toolbox
preprocess = DataPreprocessing()

In [7]:
data = preprocess.data_load(filename ='1-0-hgk-MD-ActCoeff-processed_Activity.csv')

In [8]:
data.head()

,#,Name of the polymer,CounterIon,Co-Ion,salt,Water-per-ion,concentration of salt (M),gr_minima (Ion_H2O),gr_peak_position (Ion_H2O),gr_peak_height (Ion_H2O),Nr_(Ion_H2O),gr_minima (CG_H2O),gr_peak_position (CG_H2O),gr_peak_hight (CG_H2O),Nr_(CG_H2O),gr_minima (CG_Ion),gr_peak_position (CG_Ion),gr_peak_height (CG_Ion),Nr_(CG_Ion),Experimental activity co.
0,1,PSbNMP_RCE,I-,K+,KI,6,0.02,4.45,3.65,6.50,4.70,5.50,3.55,2.30,6.10,6.15,4.15,5.80,1.90,0.300000
1,2,PSbNMP_RCE,I-,K+,KI,6,0.20,4.45,3.65,6.30,4.60,5.50,3.55,1.90,5.00,6.15,4.15,5.80,2.00,0.319000
2,3,PSbNMP_RCE,I-,K+,KI,6,0.50,4.45,3.65,6.20,4.60,5.50,3.55,1.80,4.50,6.15,4.15,5.60,2.20,0.476000
3,4,PSbNMP_RCE,I-,K+,KI,6,1.00,4.55,3.65,6.00,4.70,5.00,3.55,1.50,3.00,6.15,4.15,5.10,2.50,0.663000
4,5,PVAS_RCE,Na+,Cl-,NaCl,113,0.02,3.15,2.35,8.55,5.38,3.37,2.76,1.83,2.98,3.06,2.35,6.71,0.13,0.280899


# A0 - No transformation

In [9]:
## assign X, y
(X, y, MFprints) = preprocess.featurizer(data, polymers_dict,  'activity') # descr = 'solvation' or 'activity'

# A1 - PCA (Fingerprint) / A2 - PCA (ALL)

In [11]:
(data_activity_scaler, X_train, X_test, y_train, y_test) = preprocess.normalizedata(X, y, splitRatio=0.2, state = 48, transform = True, property = None)
print(f"X_train, X_test, y_train, & y_test has shapes of {X_train.shape}, {X_test.shape}, {y_train.shape}, & {y_test.shape} respectively.")

X_train, X_test, y_train, & y_test has shapes of (64, 153), (16, 153), (64,), & (16,) respectively.


In [12]:
# transform features using PCA
print("***********************A1*************************")
pca_X_train_MDFP, pca_X_test_MDFP = preprocess.convertInputsPCA(X_train, X_test, MFprints, n_comp = 15, type = 'MDFP')
print(f"pca_X_train_MDFP & pca_X_test_MDFP has shapes of {pca_X_train_MDFP.shape}, and {pca_X_test_MDFP.shape}, respectively.")
#
print("***********************A2*************************")
pca_X_train_ALL, pca_X_test_ALL = preprocess.convertInputsPCA(X_train, X_test, MFprints, n_comp = 15, type = 'All')
print(f"pca_X_train_ALL & pca_X_test_ALL has shapes of {pca_X_train_ALL.shape}, and {pca_X_test_ALL.shape}, respectively.")


***********************A1*************************
pca_X_train_MDFP & pca_X_test_MDFP has shapes of (64, 40), and (16, 40), respectively.
***********************A2*************************
pca_X_train_ALL & pca_X_test_ALL has shapes of (64, 15), and (16, 15), respectively.


# ML models - Hyperparameters optimization

### SVR - A2

In [47]:
solver = svrSolvers()
trainer = Trainers()

res = solver.svrSolver(X_train=pca_X_train_ALL,
                X_test=pca_X_test_ALL,
                y_train=y_train,
                y_test=y_test)

h_params, best_F = solver.get_best(res)

score = trainer.trainSVR_SingleOutput(h_params=h_params,
                        X_train=X_train,
                        X_test=X_test,
                        y_train=y_train,
                        y_test=y_test,
                        folds=None)

{'C': 41.76049826978715, 'epsilon': 0.012286352598724284}
{'C': 72.0604168948716, 'epsilon': 6.053628126910477}
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      1 |             - |             -
{'C': 72.0604168948716, 'epsilon': 0.3009392332060581}
{'C': 41.76049826978715, 'epsilon': 6.069687443092774}
{'C': 52.864167896745705, 'epsilon': 5.949025304005646}
{'C': 72.0604168948716, 'epsilon': 0.5072755760661412}
     2 |        6 |      1 |  0.000000E+00 |             f
{'C': 41.76049826978715, 'epsilon': 1.7926026381996865}
{'C': 66.10316060065897, 'epsilon': 0.3009392332060581}
{'C': 72.0604168948716, 'epsilon': 0.0298703801962584}
{'C': 78.65931170889557, 'epsilon': 0.013811636386431708}
     3 |       10 |      1 |  0.0309778916 |         ideal
{'C': 72.0604168948716, 'epsilon': 0.014091181742050613}
{'C': 76.22069036392507, 'epsilon': 0.02940404251381104}
{'C': 75.0676412559542, 'epsilon': 2.457827844596563}
{'C': 69.17206279904605, 'epsilon': 0.

In [48]:
print(h_params)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

{'C': 99.87571284612757, 'epsilon': 0.023340201888221765}


TypeError: 'SVR' object is not subscriptable

In [35]:
import pickle

In [49]:
filename = 'c:/Users/gtheis2/git/ACTC_MD_ML/models_gaby/activity-coefficient/Activityco_SVR_A2_score.pkl'
pickle.dump(score, open(filename, 'wb'))

In [ ]:
filename = 'c:/Users/gtheis2/git/ACTC_MD_ML/models_gaby/activity-coefficient/Activityco_SVR_A2.sav'
pickle.dump(h_params, open(filename, 'wb'))

In [50]:
filename = 'c:/Users/gtheis2/git/ACTC_MD_ML/models_gaby/activity-coefficient/Activityco_SVR_A2_score.pkl'
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model

SVR(C=99.87571284612757, epsilon=0.023340201888221765)

In [51]:
preprocess.plot(loaded_model,pca_X_train_ALL,y_train,pca_X_test_ALL,y_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- pca_0
- pca_1
- pca_10
- pca_11
- pca_12
- ...
Feature names seen at fit time, yet now missing:
- A
- B
- C
- D
- E
- ...


A1

In [32]:
solver1 = svrSolvers()
trainer1 = Trainers()

res1 = solver1.svrSolver(X_train=pca_X_train_MDFP,
                X_test=pca_X_test_MDFP,
                y_train=y_train,
                y_test=y_test)

h_params1, best_F1 = solver1.get_best(res1)

score = trainer1.trainSVR_SingleOutput(h_params=h_params1,
                        X_train=X_train,
                        X_test=X_test,
                        y_train=y_train,
                        y_test=y_test,
                        folds=5)

{'C': 41.76049826978715, 'epsilon': 0.012286352598724284}
{'C': 72.0604168948716, 'epsilon': 6.053628126910477}
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      1 |             - |             -
{'C': 72.0604168948716, 'epsilon': 0.3009392332060581}
{'C': 41.76049826978715, 'epsilon': 6.069687443092774}
{'C': 52.864167896745705, 'epsilon': 5.949025304005646}
{'C': 72.0604168948716, 'epsilon': 0.5072755760661412}
     2 |        6 |      1 |  0.000000E+00 |             f
{'C': 41.76049826978715, 'epsilon': 1.7926026381996865}
{'C': 66.10316060065897, 'epsilon': 0.3009392332060581}
{'C': 72.0604168948716, 'epsilon': 0.0298703801962584}
{'C': 78.65931170889557, 'epsilon': 0.013811636386431708}
     3 |       10 |      1 |  0.0203557131 |         ideal
{'C': 71.62019013087945, 'epsilon': 1.1894834012560185}
{'C': 84.36311326889772, 'epsilon': 0.015036048508410867}
{'C': 73.19186516411669, 'epsilon': 1.8941337826484623}
{'C': 71.4474218719919, 'epsilon': 1

In [29]:
print(h_params1)
print("A1 is %0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

{'C': 99.99364647653165, 'epsilon': 0.021861420689992098}
A1 is 0.95 accuracy with a standard deviation of 0.03. R^2: 0.87


A0

In [24]:
solver2 = svrSolvers()
trainer2 = Trainers()

res2 = solver2.svrSolver(X_train=X_train,
                X_test=X_test,
                y_train=y_train,
                y_test=y_test)

h_params2, best_F2 = solver1.get_best(res2)

score = trainer1.trainSVR_SingleOutput(h_params=h_params2,
                        X_train=X_train,
                        X_test=X_test,
                        y_train=y_train,
                        y_test=y_test,
                        folds=5)

{'C': 41.76049826978715, 'epsilon': 0.012286352598724284}
{'C': 72.0604168948716, 'epsilon': 6.053628126910477}
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      1 |             - |             -
{'C': 72.0604168948716, 'epsilon': 0.3009392332060581}
{'C': 41.76049826978715, 'epsilon': 6.069687443092774}
{'C': 52.864167896745705, 'epsilon': 5.949025304005646}
{'C': 72.0604168948716, 'epsilon': 0.5072755760661412}
     2 |        6 |      1 |  0.000000E+00 |             f
{'C': 41.76049826978715, 'epsilon': 1.7926026381996865}
{'C': 66.10316060065897, 'epsilon': 0.3009392332060581}
{'C': 72.0604168948716, 'epsilon': 0.0298703801962584}
{'C': 78.65931170889557, 'epsilon': 0.013811636386431708}
     3 |       10 |      1 |  0.0226168543 |         ideal
{'C': 71.62019013087945, 'epsilon': 1.1894834012560185}
{'C': 84.36311326889772, 'epsilon': 0.015036048508410867}
{'C': 73.19186516411669, 'epsilon': 1.8941337826484623}
{'C': 71.4474218719919, 'epsilon': 1

In [27]:
print(h_params2)
print("A0 is %0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

{'C': 95.98983019144184, 'epsilon': 0.021496048288544144}
A0 is
Ao is 0.95 accuracy with a standard deviation of 0.03. R^2: 0.87
